# Avaliação de Modelos

In [1]:
import tensorflow as tf
from funcoes import getItem, gen_batch

import numpy as np
import matplotlib.pyplot as plt
import os, argparse, pathlib
from eval import eval

In [2]:
# Definicao de Parametros
trainFile = 'train_split.txt'
valFile = 'val_split.txt'
testFile = 'test_split.txt'
datadir = '..\\data\\'
trainDir = 'train'
valDir = 'train'
testDir = 'test'
covid_percent= 0.3
input_size = 480
top_percent = 0.08
batch_size = 10
modelo = 'covid_5ep_cnn_model.h5'

In [3]:
generator_test = gen_batch(data_dir = datadir,
              tipo_dir = testDir,
              csv_file = testFile,
              batch_size = batch_size,
              input_shape = (input_size, input_size),
              num_channels = 3,
              covid_percent = covid_percent,
              top_percent = top_percent)

In [4]:
model = tf.keras.models.load_model(modelo)

In [5]:
test_loss, test_acc = model.evaluate(generator_test, steps = 147)

147
147/147 [==============================] - 31s 211ms/step - loss: 0.8509 - accuracy: 0.6259


In [6]:
con_mat = np.zeros((3,3))
pred = np.array([])
labels = np.array([])
for item in range(0,147):
    testX, testY = next(generator_test)
    aux = model.predict(testX)
    pred = np.concatenate((pred,np.argmax(aux, axis=1)))
    labels = np.concatenate((labels,testY))

In [7]:
from sklearn.metrics import classification_report, confusion_matrix

In [8]:
print(classification_report(labels,pred))

              precision    recall  f1-score   support

         0.0       0.60      0.90      0.72       608
         1.0       0.72      0.49      0.58       421
         2.0       0.66      0.41      0.50       441

    accuracy                           0.63      1470
   macro avg       0.66      0.60      0.60      1470
weighted avg       0.65      0.63      0.62      1470



In [9]:
print(confusion_matrix(labels,pred))

[[548  41  19]
 [141 205  75]
 [222  39 180]]


In [10]:
con_mat = confusion_matrix(labels,pred)
sensibilidade = con_mat[2,2] / (con_mat[2,2] + con_mat[2,0] + con_mat[2,1])
sensibilidade

0.40816326530612246

In [11]:
PPV = con_mat[2,2] / (con_mat[2,2] + con_mat[0,2] + con_mat[1,2])
PPV

0.656934306569343